# OVOS Intent Classification — Monolingual Model
## Training, ONNX Export & Dependency-Free Inference

This notebook covers the full pipeline for a **single-language** subset of the OVOS intents dataset:
1. **Stream & merge** all splits (train / validation / test) from HuggingFace
2. **Filter** by the target language
3. **Create balanced** stratified train / validation / test splits
4. **Train** classifiers using lightweight `model2vec` static models
5. **Export** to ONNX (embeddings + MLP head in one file)
6. **Inference** using only `onnxruntime` + `numpy` (no `model2vec` / `torch` / `tokenizers`)

**Dataset:** [`OpenVoiceOS/ovos-intents-train-latest`](https://huggingface.co/datasets/OpenVoiceOS/ovos-intents-train-latest)
**Subset:** Single language, configured via `LANG` environment variable
**Task:** Multi-class intent classification

---

> **Credits:** Funded through the [NGI0 Commons Fund](https://nlnet.nl/commonsfund) via [NLnet](https://nlnet.nl), with support from the European Commission's Next Generation Internet programme (grant No 101135429).

---
## 0. Installs & Imports

In [11]:
!pip install model2vec[train] datasets torch onnx onnxruntime onnxscript numpy scikit-learn

Using Python 3.11.14 environment at: /home/miro/PycharmProjects/wakeHuBert/.venv
Audited 8 packages in 24ms


In [12]:
import gc
import os
import re
import json
import random
import datetime
import unicodedata
from pathlib import Path
from collections import Counter

import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from model2vec.train import StaticModelForClassification

---
## 1. Configuration

Every parameter is read from an environment variable with a sensible default.
Override any of them from your shell, a `.env` file, or notebook `%env` magic
**before** running this cell.

| Variable | Default | Description |
|---|---|---|
| `DATASET_NAME` | `OpenVoiceOS/ovos-intents-train-latest` | HuggingFace dataset identifier |
| `LANG` | `en` | ISO-639-1 language code to keep |
| `SENTENCE_COL` | `sentence` | Column containing the input text |
| `LABEL_COL` | `label` | Column containing the intent label |
| `LANG_COL` | `lang` | Column containing the language tag |
| `TEST_RATIO` | `0.15` | Fraction held out for testing |
| `VAL_RATIO` | `0.15` | Fraction held out for validation |
| `SEED` | `42` | Random seed for reproducibility |
| `MAX_EPOCHS` | `25` | Maximum training epochs |

In [13]:
# ── Read from environment (with defaults) ──────────────────────
DATASET_NAME  = os.environ.get("DATASET_NAME", "OpenVoiceOS/MT-intents-dataset-pt-PT")
LANG          = "pt-PT"
SENTENCE_COL  = os.environ.get("SENTENCE_COL", "sentence")
LABEL_COL     = os.environ.get("LABEL_COL", "label")
LANG_COL      = os.environ.get("LANG_COL", "lang")
TEST_RATIO    = float(os.environ.get("TEST_RATIO", "0.15"))
VAL_RATIO     = float(os.environ.get("VAL_RATIO", "0.15"))
SEED          = int(os.environ.get("SEED", "42"))
MAX_EPOCHS    = int(os.environ.get("MAX_EPOCHS", "25"))

EXPORT_DIR = Path(f"exported_models_{LANG}")
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

MODELS = [
        "Jarbas/m2v-256-paraphrase-multilingual-MiniLM-L12-v2",
        "minishlab/potion-multilingual-128M",
        "Jarbas/m2v-256-bert-base-portuguese-cased",
        "Jarbas/m2v-256-bert-large-portuguese-cased",
        "Jarbas/m2v-256-albertina-100m-portuguese-ptpt-encoder",
        "Jarbas/m2v-256-albertina-100m-portuguese-ptbr-encoder",
        "Jarbas/m2v-256-serafim-100m-portuguese-pt-sentence-encoder",
        "Jarbas/m2v-256-serafim-335m-portuguese-pt-sentence-encoder",
]

RESULTS = {}

print(f"Dataset:       {DATASET_NAME}")
print(f"Language:      {LANG}")
print(f"Columns:       sentence={SENTENCE_COL!r}  label={LABEL_COL!r}  lang={LANG_COL!r}")
print(f"Split ratios:  test={TEST_RATIO}  val={VAL_RATIO}  train={1 - TEST_RATIO - VAL_RATIO:.2f}")
print(f"Seed:          {SEED}")
print(f"Max epochs:    {MAX_EPOCHS}")
print(f"Models:        {MODELS}")
print(f"Export dir:    {EXPORT_DIR}")

Dataset:       OpenVoiceOS/MT-intents-dataset-pt-PT
Language:      pt-PT
Columns:       sentence='sentence'  label='label'  lang='lang'
Split ratios:  test=0.15  val=0.15  train=0.70
Seed:          42
Max epochs:    25
Models:        ['Jarbas/m2v-256-bert-base-portuguese-cased', 'Jarbas/m2v-256-bert-large-portuguese-cased', 'Jarbas/m2v-256-albertina-100m-portuguese-ptpt-encoder', 'Jarbas/m2v-256-albertina-100m-portuguese-ptbr-encoder', 'Jarbas/m2v-256-serafim-100m-portuguese-pt-sentence-encoder', 'Jarbas/m2v-256-serafim-335m-portuguese-pt-sentence-encoder', 'Jarbas/m2v-256-paraphrase-multilingual-MiniLM-L12-v2', 'minishlab/potion-multilingual-128M']
Export dir:    exported_models_pt-PT


---
## 2. Stream, Merge & Filter by Language

The upstream dataset ships pre-made train / validation / test splits, but they
are **not balanced across labels**.  We therefore:

1. **Stream** every split into memory (plain Python lists — no HF cache bloat).
2. **Concatenate** all rows into a single pool.
3. **Filter** to keep only the target `LANG`.

This gives us maximum data for the language and lets us create our own balanced
split in the next step.

In [14]:
def stream_split(dataset_name: str, split: str) -> list[dict]:
    """Stream a single split into a list of dicts (one per row)."""
    rows = []
    ds = load_dataset(dataset_name, split=split, streaming=True)
    for row in ds:
        rows.append({
            "sentence": row[SENTENCE_COL],
            "label":    row[LABEL_COL],
            "lang":     row[LANG_COL],
        })
    return rows


print("Streaming all splits …")
all_rows = []
for split_name in ("train", "validation", "test"):
    split_rows = stream_split(DATASET_NAME, split_name)
    print(f"  {split_name}: {len(split_rows)} rows")
    all_rows.extend(split_rows)

print(f"\nTotal rows (all languages): {len(all_rows)}")

# ── Filter by language ─────────────────────────────────────────
lang_rows = [r for r in all_rows if r["lang"] == LANG]
print(f"Rows for lang={LANG!r}: {len(lang_rows)}")

# Free the full pool
del all_rows
gc.collect()

if len(lang_rows) == 0:
    raise RuntimeError(
        f"No rows found for lang={LANG!r}. "
        f"Available languages in first split: check the dataset card."
    )

# Quick stats
label_counts = Counter(r["label"] for r in lang_rows)
print(f"Unique labels: {len(label_counts)}")
print(f"Label distribution (top-10):")
for lbl, cnt in label_counts.most_common(10):
    print(f"  {lbl}: {cnt}")

Streaming all splits …
  train: 9515 rows
  validation: 2472 rows
  test: 157 rows

Total rows (all languages): 12144
Rows for lang='pt-PT': 12144
Unique labels: 157
Label distribution (top-10):
  ocp:play: 1874
  ovos-skill-weather.openvoiceos:hourly_forecast.intent: 1574
  common_query:common_query: 1099
  ovos-skill-weather.openvoiceos:daily_forecast.intent: 480
  ovos-skill-weather.openvoiceos:hourly_temperature.intent: 476
  ovos-skill-date-time.openvoiceos:what.time.will.it.be.intent: 421
  ovos-skill-date-time.openvoiceos:what.time.is.it.intent: 338
  ovos-skill-wordnet.openvoiceos:definition.intent: 329
  ovos-skill-weather.openvoiceos:high_temperature.intent: 269
  ovos-skill-date-time.openvoiceos:date.future.weekend.intent: 265


---
## 3. Balanced Stratified Split

We perform a **stratified** split that preserves the label distribution across
train / validation / test.  Rare labels (fewer than 3 samples) are placed
entirely in the training set to avoid empty folds.

In [15]:
random.seed(SEED)
np.random.seed(SEED)

def stratified_split(
    rows: list[dict],
    test_ratio: float,
    val_ratio: float,
    seed: int,
) -> tuple[dict, dict, dict]:
    """Stratified split into {sentence, label} dicts.

    Labels with fewer than 3 samples go entirely to train.
    """
    rng = random.Random(seed)

    # Group by label
    by_label: dict[str, list[dict]] = {}
    for r in rows:
        by_label.setdefault(r["label"], []).append(r)

    train_rows, val_rows, test_rows = [], [], []

    for label, items in by_label.items():
        rng.shuffle(items)
        n = len(items)
        if n < 3:
            # Too few samples — put everything in train
            train_rows.extend(items)
            continue
        n_test = max(1, int(round(n * test_ratio)))
        n_val  = max(1, int(round(n * val_ratio)))
        # Clamp so we always have at least 1 training sample
        if n_test + n_val >= n:
            n_test = max(1, n // 3)
            n_val  = max(1, n // 3)
        test_rows.extend(items[:n_test])
        val_rows.extend(items[n_test : n_test + n_val])
        train_rows.extend(items[n_test + n_val :])

    # Final shuffle within each split
    rng.shuffle(train_rows)
    rng.shuffle(val_rows)
    rng.shuffle(test_rows)

    def to_dict(rows):
        return {
            "sentence": [r["sentence"] for r in rows],
            "label":    [r["label"]    for r in rows],
        }

    return to_dict(train_rows), to_dict(val_rows), to_dict(test_rows)


train_data, val_data, test_data = stratified_split(
    lang_rows, TEST_RATIO, VAL_RATIO, SEED
)

# Free filtered pool
del lang_rows
gc.collect()

print(f"Train:      {len(train_data['sentence']):>6}")
print(f"Validation: {len(val_data['sentence']):>6}")
print(f"Test:       {len(test_data['sentence']):>6}")
print(f"Labels:     {len(set(train_data['label']))} unique in train")

# Verify balance
train_counts = Counter(train_data["label"])
test_counts  = Counter(test_data["label"])
val_counts   = Counter(val_data["label"])
print(f"\nLabel coverage — train: {len(train_counts)}  val: {len(val_counts)}  test: {len(test_counts)}")

# Show a sample
print(f"\nSample: sentence={train_data['sentence'][0]!r}")
print(f"        label={train_data['label'][0]!r}")

Train:        8488
Validation:   1828
Test:         1828
Labels:     157 unique in train

Label coverage — train: 157  val: 153  test: 153

Sample: sentence='como está o tempo em {location} na manhã de segunda-feira'
        label='ovos-skill-weather.openvoiceos:hourly_forecast.intent'


---
## 4. Helpers

In [16]:
def parse_classification_report(report_str: str) -> dict:
    lines = [l.strip() for l in report_str.split("\n") if l.strip()]
    data = {}
    header_re = re.compile(r"precision\s+recall\s+f1-score\s+support")
    row_re = re.compile(
        r"^(?P<label>[\w\s\-\.:]+?)\s+"
        r"(?P<precision>\d\.\d+|\d)\s+"
        r"(?P<recall>\d\.\d+|\d)\s+"
        r"(?P<f1>\d\.\d+|\d)\s+"
        r"(?P<support>\d+)$"
    )
    acc_re = re.compile(r"^accuracy\s+(?P<precision>[\s\d\.]+)\s+(?P<support>\d+)$")
    header_found = False
    for line in lines:
        if header_re.search(line):
            header_found = True
            continue
        if not header_found:
            continue
        m = row_re.match(line)
        if m:
            label = m.group("label").strip()
            data[label] = {
                "precision": float(m.group("precision")),
                "recall": float(m.group("recall")),
                "f1-score": float(m.group("f1")),
                "support": int(m.group("support")),
            }
            continue
        m = acc_re.match(line)
        if m:
            data["accuracy"] = float(m.group("precision"))
            data["accuracy_support"] = int(m.group("support"))
    return data

---
## 5. ONNX Export Utilities

In [17]:
class OnnxExportWrapper(nn.Module):
    """Thin wrapper: takes padded input_ids, returns logits only."""

    def __init__(self, classifier: StaticModelForClassification):
        super().__init__()
        self.classifier = classifier

    def forward(self, input_ids: torch.Tensor) -> torch.Tensor:
        logits, _ = self.classifier(input_ids)
        return logits

In [18]:
def export_classifier_to_onnx(
    classifier: StaticModelForClassification,
    save_dir: Path,
) -> None:
    save_dir.mkdir(parents=True, exist_ok=True)
    classifier.eval()

    wrapper = OnnxExportWrapper(classifier)
    wrapper.eval()

    dummy_input = classifier.tokenize(["hello", "hello world"])
    print(f"  Dummy input shape: {dummy_input.shape}  (batch, max_seq_len)")

    onnx_path = save_dir / "classifier.onnx"
    torch.onnx.export(
        wrapper,
        (dummy_input,),
        str(onnx_path),
        export_params=True,
        opset_version=18,
        do_constant_folding=True,
        input_names=["input_ids"],
        output_names=["logits"],
        dynamic_axes={
            "input_ids": {0: "batch_size", 1: "seq_len"},
            "logits": {0: "batch_size"},
        },
    )

    config = {
        "labels": list(classifier.classes_),
        "pad_id": int(classifier.pad_id),
        "multilabel": bool(classifier.multilabel),
    }
    (save_dir / "config.json").write_text(json.dumps(config, indent=2))
    classifier.tokenizer.save(str(save_dir / "tokenizer.json"))

    print(f"  Exported: {onnx_path}")
    print(f"  Labels:   {len(config['labels'])} classes")
    print(f"  Pad ID:   {config['pad_id']}")
    print(f"  Files:    classifier.onnx, config.json, tokenizer.json")

---
## 6. Train & Export Loop

Each model is loaded, trained on the **monolingual** train set, evaluated on
the held-out test set, and exported to ONNX.

Memory is freed between models so only one set of weights is resident at a
time.

In [19]:
for model_name in MODELS:
    print(f"\n{'='*60}")
    print(f"Model: {model_name}")
    print(f"{'='*60}")

    classifier = StaticModelForClassification.from_pretrained(model_name=model_name)
    classifier.fit(
        max_epochs=MAX_EPOCHS,
        X=train_data["sentence"],
        y=train_data["label"],
        X_val=val_data["sentence"],
        y_val=val_data["label"],
    )

    raw_report = classifier.evaluate(test_data["sentence"], test_data["label"])
    print(raw_report)
    parsed = parse_classification_report(raw_report)

    short_name = model_name.split("/")[-1]
    pipeline_name = f"ovos-intents-{LANG}-{short_name}"
    pipeline = classifier.to_pipeline()
    pipeline.save_pretrained(pipeline_name)

    onnx_dir = EXPORT_DIR / short_name
    print(f"\nExporting to ONNX …")
    export_classifier_to_onnx(classifier, onnx_dir)

    RESULTS[model_name] = {
        "raw": raw_report,
        "parsed": parsed,
        "onnx_dir": str(onnx_dir),
    }

    # Free memory before next model
    del classifier, pipeline
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\nAll models trained and exported!")


Model: Jarbas/m2v-256-bert-base-portuguese-cased


Your vectors are torch.float16 precision, converting to to torch.float32 to avoid compatibility issues.
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type                         ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model         │ StaticModelForClassification │  7.9 M │ train │     0 │
│ 1 │ loss_function │ CrossEntropyLoss             │      0 │ train │     0 │
└───┴───────────────┴──────────────────────────────┴────────┴───────┴───────┘

Trainable params: 7.8 M                                                                                            
Non-trainable params: 29.7 K                                                                                       
Total params: 7.9 M                                                                                                
Total estimated model params size (MB): 31                                                                         
Modules in train mode: 7                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

...

                                                                              precision    recall  f1-score   support

                                                   common_query:common_query       0.90      0.93      0.91       165
                                                                    ocp:play       0.96      0.96      0.96       281
                               ovos-skill-alerts.openvoiceos:AddListSubitems       1.00      1.00      1.00         2
                                  ovos-skill-alerts.openvoiceos:CalendarList       1.00      1.00      1.00         1
                                   ovos-skill-alerts.openvoiceos:CancelAlert       1.00      0.50      0.67         2
                              ovos-skill-alerts.openvoiceos:ChangeProperties       0.00      0.00      0.00         1
                                   ovos-skill-alerts.openvoiceos:CreateAlarm       0.50      0.50      0.50         2
                                   ovos-skill-alerts.op

/tmp/ipykernel_30966/2828041513.py:15: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0216 17:51:46.877000 30966 torch/onnx/_internal/exporter/_registration.py:110] torchvision is not installed. Skipping torchvision::nms
W0216 17:51:46.879000 30966 torch/onnx/_internal/exporter/_registration.py:110] torchvision is not installed. Skipping torchvision::roi_align
W0216 17:51:46.881000 30966 torch/onnx/_internal/exporter/_registration.py:110] torchvision is not installed. Skipping torchvision::roi_pool


[torch.onnx] Obtain model graph for `OnnxExportWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `OnnxExportWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/home/miro/.local/share/uv/python/cpython-3.11.14-linux-x86_64-gnu/lib/python3.11/copyreg.py:105: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)


[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 7 of general pattern rewrite rules.
  Exported: exported_models_pt-PT/potion-multilingual-128M/classifier.onnx
  Labels:   157 classes
  Pad ID:   0
  Files:    classifier.onnx, config.json, tokenizer.json

All models trained and exported!


---
## 7. Results Summary

In [20]:
print(f"{'Model':<40} {'Accuracy':>10} {'F1 (macro)':>12}")
print("-" * 65)
for model_name, result in RESULTS.items():
    parsed = result["parsed"]
    acc = parsed.get("accuracy", 0)
    macro_f1 = parsed.get("macro avg", {}).get("f1-score", 0)
    print(f"{model_name:<40} {acc:>10.4f} {macro_f1:>12.4f}")

Model                                      Accuracy   F1 (macro)
-----------------------------------------------------------------
Jarbas/m2v-256-bert-base-portuguese-cased     0.9400       0.8100
Jarbas/m2v-256-bert-large-portuguese-cased     0.9400       0.7900
Jarbas/m2v-256-albertina-100m-portuguese-ptpt-encoder     0.9300       0.7600
Jarbas/m2v-256-albertina-100m-portuguese-ptbr-encoder     0.9300       0.7500
Jarbas/m2v-256-serafim-100m-portuguese-pt-sentence-encoder     0.9500       0.8300
Jarbas/m2v-256-serafim-335m-portuguese-pt-sentence-encoder     0.9500       0.8300
Jarbas/m2v-256-paraphrase-multilingual-MiniLM-L12-v2     0.9400       0.8000
minishlab/potion-multilingual-128M           0.9500       0.8300


---
# Part 2 — Dependency-Free ONNX Inference

Everything below runs with **only** `onnxruntime` + `numpy` (+ the standard
library).  No `model2vec`, `torch`, or `tokenizers` needed.

---
## 8. Pure-Python Tokenizer (auto-detects WordPiece / BPE / Unigram)

Reads the HuggingFace `tokenizer.json` and auto-detects the model type.
Supports the three main tokenizer types used by model2vec models:
- **WordPiece** (BERT-style)
- **BPE** (GPT-style)
- **Unigram** (SentencePiece-style)

In [21]:
import json
import math
import unicodedata
import numpy as np
import onnxruntime as ort


class MinimalTokenizer:
    """
    Pure-Python tokenizer that reads a HuggingFace tokenizer.json.
    Auto-detects WordPiece, BPE, or Unigram model type.
    No dependencies beyond the standard library + numpy.
    """

    def __init__(self, tokenizer_json_path: str):
        with open(tokenizer_json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        self.model_type = data["model"]["type"]  # "WordPiece", "BPE", "Unigram"
        self.model_data = data["model"]
        self.normalizer_config = data.get("normalizer")
        self.pre_tokenizer_config = data.get("pre_tokenizer")

        # Build vocab lookup based on model type
        if self.model_type == "Unigram":
            self._init_unigram()
        elif self.model_type == "WordPiece":
            self._init_wordpiece()
        elif self.model_type == "BPE":
            self._init_bpe()
        else:
            raise ValueError(f"Unsupported tokenizer model type: {self.model_type}")

    # ================================================================
    # UNIGRAM (SentencePiece)
    # ================================================================
    def _init_unigram(self):
        vocab_list = self.model_data["vocab"]
        self.unk_id = self.model_data.get("unk_id", 0)
        self.vocab = {}
        for idx, (piece, score) in enumerate(vocab_list):
            self.vocab[piece] = (idx, score)
        self.max_piece_len = max(len(p) for p, _ in vocab_list)

    def _tokenize_unigram(self, text: str) -> list[int]:
        """Viterbi decoding for Unigram tokenization."""
        n = len(text)
        if n == 0:
            return []
        NEG_INF = float("-inf")
        best_score = [NEG_INF] * (n + 1)
        best_prev = [0] * (n + 1)
        best_score[0] = 0.0

        for i in range(n):
            if best_score[i] == NEG_INF:
                continue
            max_len = min(self.max_piece_len, n - i)
            for length in range(1, max_len + 1):
                piece = text[i : i + length]
                if piece in self.vocab:
                    _, score = self.vocab[piece]
                    new_score = best_score[i] + score
                    if new_score > best_score[i + length]:
                        best_score[i + length] = new_score
                        best_prev[i + length] = i

        if best_score[n] == NEG_INF:
            return self._unigram_fallback(text)

        ids = []
        pos = n
        while pos > 0:
            prev = best_prev[pos]
            piece = text[prev:pos]
            token_id, _ = self.vocab[piece]
            ids.append(token_id)
            pos = prev
        ids.reverse()
        return ids

    def _unigram_fallback(self, text: str) -> list[int]:
        ids = []
        for ch in text:
            if ch in self.vocab:
                ids.append(self.vocab[ch][0])
            else:
                ids.append(self.unk_id)
        return ids

    # ================================================================
    # WORDPIECE
    # ================================================================
    def _init_wordpiece(self):
        self.vocab = self.model_data["vocab"]
        self.unk_token = self.model_data.get("unk_token", "[UNK]")
        self.unk_id = self.vocab.get(self.unk_token)
        self.continuing_subword_prefix = self.model_data.get(
            "continuing_subword_prefix", "##"
        )
        self.max_input_chars_per_word = self.model_data.get(
            "max_input_chars_per_word", 100
        )

    def _wordpiece_single(self, word: str) -> list[int]:
        if len(word) > self.max_input_chars_per_word:
            return [self.unk_id] if self.unk_id is not None else []
        token_ids = []
        start = 0
        while start < len(word):
            end = len(word)
            found = False
            while start < end:
                substr = word[start:end]
                if start > 0:
                    substr = self.continuing_subword_prefix + substr
                if substr in self.vocab:
                    token_ids.append(self.vocab[substr])
                    found = True
                    break
                end -= 1
            if not found:
                return [self.unk_id] if self.unk_id is not None else []
            start = end
        return token_ids

    # ================================================================
    # BPE
    # ================================================================
    def _init_bpe(self):
        self.vocab = self.model_data["vocab"]
        self.merges = self.model_data.get("merges", [])
        self.unk_token = self.model_data.get("unk_token", "<unk>")
        self.unk_id = self.vocab.get(self.unk_token)
        self.merge_ranks = {}
        for rank, merge_str in enumerate(self.merges):
            parts = merge_str.split(" ", 1)
            if len(parts) == 2:
                self.merge_ranks[(parts[0], parts[1])] = rank

    def _bpe_single(self, word: str) -> list[int]:
        symbols = list(word)
        while len(symbols) > 1:
            best_pair = None
            best_rank = float("inf")
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i + 1])
                rank = self.merge_ranks.get(pair, float("inf"))
                if rank < best_rank:
                    best_rank = rank
                    best_pair = pair
            if best_pair is None or best_rank == float("inf"):
                break
            merged = best_pair[0] + best_pair[1]
            new_symbols = []
            i = 0
            while i < len(symbols):
                if (
                    i < len(symbols) - 1
                    and symbols[i] == best_pair[0]
                    and symbols[i + 1] == best_pair[1]
                ):
                    new_symbols.append(merged)
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols
        return [
            self.vocab.get(s, self.unk_id if self.unk_id is not None else 0)
            for s in symbols
        ]

    # ================================================================
    # NORMALIZER
    # ================================================================
    def _normalize(self, text: str) -> str:
        if not self.normalizer_config:
            return text
        return self._apply_normalizer(text, self.normalizer_config)

    def _apply_normalizer(self, text: str, cfg: dict) -> str:
        ntype = cfg.get("type", "")

        if ntype == "Sequence":
            for sub in cfg.get("normalizers", []):
                text = self._apply_normalizer(text, sub)
            return text

        if ntype == "Precompiled":
            text = unicodedata.normalize("NFKC", text)
            return text

        if ntype == "Replace":
            pattern = cfg.get("pattern", {})
            content = cfg.get("content", "")
            if "String" in pattern:
                text = text.replace(pattern["String"], content)
            elif "Regex" in pattern:
                text = re.sub(pattern["Regex"], content, text)
            return text

        if ntype == "NFKC":
            return unicodedata.normalize("NFKC", text)

        if ntype in ("BertNormalizer", "Lowercase"):
            if cfg.get("lowercase", True):
                text = text.lower()
            if cfg.get("strip_accents", False):
                text = "".join(
                    c
                    for c in unicodedata.normalize("NFD", text)
                    if unicodedata.category(c) != "Mn"
                )
            return text

        if ntype == "NFC":
            return unicodedata.normalize("NFC", text)

        if ntype == "StripAccents":
            return "".join(
                c
                for c in unicodedata.normalize("NFD", text)
                if unicodedata.category(c) != "Mn"
            )

        if ntype == "Strip":
            return text.strip()

        return text

    # ================================================================
    # PRE-TOKENIZER
    # ================================================================
    def _pre_tokenize(self, text: str) -> list[str]:
        if not self.pre_tokenizer_config:
            return [text]
        return self._apply_pre_tokenizer(text, self.pre_tokenizer_config)

    def _apply_pre_tokenizer(self, text: str, cfg: dict) -> list[str]:
        ptype = cfg.get("type", "")

        if ptype == "Sequence":
            chunks = [text]
            for sub in cfg.get("pretokenizers", []):
                new_chunks = []
                for chunk in chunks:
                    new_chunks.extend(self._apply_pre_tokenizer(chunk, sub))
                chunks = new_chunks
            return chunks

        if ptype == "Metaspace":
            replacement = cfg.get("replacement", "\u2581")
            prepend = cfg.get("prepend_scheme", "always")
            text = text.replace(" ", replacement)
            if prepend in ("always", "first"):
                if not text.startswith(replacement):
                    text = replacement + text
            return [text]

        if ptype == "WhitespaceSplit":
            return text.split()

        if ptype == "BertPreTokenizer":
            return self._pre_tokenize_whitespace_punct(text)

        if ptype == "Split":
            pattern = cfg.get("pattern", {})
            if "Regex" in pattern:
                parts = re.split(pattern["Regex"], text)
                return [p for p in parts if p]
            return [text]

        return [text]

    def _pre_tokenize_whitespace_punct(self, text: str) -> list[str]:
        tokens = []
        current = []
        for ch in text:
            if ch.isspace():
                if current:
                    tokens.append("".join(current))
                    current = []
            elif _is_punctuation(ch):
                if current:
                    tokens.append("".join(current))
                    current = []
                tokens.append(ch)
            else:
                current.append(ch)
        if current:
            tokens.append("".join(current))
        return tokens

    # ================================================================
    # PUBLIC API
    # ================================================================
    def encode(self, text: str) -> list[int]:
        text = self._normalize(text)
        chunks = self._pre_tokenize(text)

        ids = []
        if self.model_type == "Unigram":
            for chunk in chunks:
                ids.extend(self._tokenize_unigram(chunk))
        elif self.model_type == "WordPiece":
            for chunk in chunks:
                words = self._pre_tokenize_whitespace_punct(chunk)
                for word in words:
                    ids.extend(self._wordpiece_single(word))
        elif self.model_type == "BPE":
            for chunk in chunks:
                words = self._pre_tokenize_whitespace_punct(chunk)
                for word in words:
                    ids.extend(self._bpe_single(word))
        return ids

    def encode_batch(
        self, texts: list[str], max_length: int = 512
    ) -> list[list[int]]:
        return [self.encode(t)[:max_length] for t in texts]

    def encode_and_pad(
        self,
        texts: list[str],
        pad_id: int = 0,
        max_length: int = 512,
    ) -> np.ndarray:
        encoded = self.encode_batch(texts, max_length=max_length)
        max_len = max((len(ids) for ids in encoded), default=1)
        max_len = max(max_len, 1)
        padded = np.full((len(encoded), max_len), pad_id, dtype=np.int64)
        for i, ids in enumerate(encoded):
            padded[i, : len(ids)] = ids
        return padded


def _is_punctuation(ch: str) -> bool:
    cp = ord(ch)
    if (33 <= cp <= 47) or (58 <= cp <= 64) or (91 <= cp <= 96) or (123 <= cp <= 126):
        return True
    return unicodedata.category(ch).startswith("P")

---
## 9. ONNX Classifier (inference-only)

In [22]:
class OnnxClassifier:
    """
    Full inference pipeline using only onnxruntime + numpy.

    Expected directory layout:
        model_dir/
            classifier.onnx
            config.json       (labels, pad_id, multilabel)
            tokenizer.json    (HuggingFace tokenizers format)
    """

    def __init__(self, model_dir: str):
        model_dir = Path(model_dir)
        self.session = ort.InferenceSession(str(model_dir / "classifier.onnx"))

        with open(model_dir / "config.json") as f:
            config = json.load(f)
        self.labels = config["labels"]
        self.pad_id = config["pad_id"]
        self.multilabel = config.get("multilabel", False)

        self.tokenizer = MinimalTokenizer(str(model_dir / "tokenizer.json"))

    def _run(self, sentences: list[str], max_length: int = 512) -> np.ndarray:
        input_ids = self.tokenizer.encode_and_pad(
            sentences, pad_id=self.pad_id, max_length=max_length
        )
        logits = self.session.run(
            ["logits"], {"input_ids": input_ids}
        )[0]
        return logits

    def predict(
        self,
        sentences: list[str],
        max_length: int = 512,
        threshold: float = 0.5,
    ) -> list:
        logits = self._run(sentences, max_length)
        if self.multilabel:
            probs = _sigmoid(logits)
            return [
                [self.labels[j] for j, v in enumerate(row) if v > threshold]
                for row in probs
            ]
        else:
            indices = np.argmax(logits, axis=1)
            return [self.labels[i] for i in indices]

    def predict_proba(
        self, sentences: list[str], max_length: int = 512
    ) -> list[dict[str, float]]:
        logits = self._run(sentences, max_length)
        probs = _sigmoid(logits) if self.multilabel else _softmax(logits)
        return [
            {label: float(p) for label, p in zip(self.labels, row)}
            for row in probs
        ]


def _softmax(x: np.ndarray) -> np.ndarray:
    e = np.exp(x - x.max(axis=1, keepdims=True))
    return e / e.sum(axis=1, keepdims=True)


def _sigmoid(x: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-x))

---
## 10. Test ONNX Inference

In [23]:
# Use the last (larger) model for the demo
test_model_name = MODELS[-1]
test_onnx_dir = EXPORT_DIR / test_model_name.split("/")[-1]

print(f"Loading ONNX classifier from: {test_onnx_dir}")
clf = OnnxClassifier(str(test_onnx_dir))

# Grab a few samples from the test set for a quick sanity check
test_sentences = test_data["sentence"][:10]

print("\nPredictions:")
predictions = clf.predict(test_sentences)
for sent, pred in zip(test_sentences, predictions):
    print(f"  [{pred}]  {sent}")

print("\nTop-3 Probabilities:")
probs = clf.predict_proba(test_sentences[:3])
for sent, prob_dict in zip(test_sentences[:3], probs):
    top3 = sorted(prob_dict.items(), key=lambda x: -x[1])[:3]
    print(f"  {sent}")
    for label, p in top3:
        print(f"    {label}: {p:.4f}")

Loading ONNX classifier from: exported_models_pt-PT/potion-multilingual-128M

Predictions:
  [common_query:common_query]  o que é um imigrante
  [ovos-skill-weather.openvoiceos:hourly_forecast.intent]  qual é a previsão para quarta-feira às 3 da manhã em {location}
  [ovos-skill-weather.openvoiceos:hourly_temperature.intent]  Qual é a temperatura às 2 da tarde de quinta-feira
  [ocp:play]  poderia ligar o rádio AM
  [ovos-skill-weather.openvoiceos:high_temperature.intent]  Qual é a temperatura máxima amanhã à noite
  [ovos-skill-weather.openvoiceos:hourly_forecast.intent]  Qual é a previsão para segunda-feira às 2 da tarde
  [ocp:play]  carregue o doctor demento e toque-o na minha cama
  [ovos-skill-weather.openvoiceos:hourly_temperature.intent]  quão frio vai ficar esta noite?
  [ovos-skill-date-time.openvoiceos:date.last.weekend.intent]  Quais foram as datas do fim de semana anterior
  [ovos-skill-parrot.openvoiceos:did.you.hear.me.intent]  ouves isto?

Top-3 Probabilities:
  o que é

---
## 11. Verify Tokenizer Alignment

In [24]:
# Compare: HuggingFace tokenizer vs MinimalTokenizer
try:
    from model2vec.train import StaticModelForClassification
    check_model = MODELS[-1]
    check_classifier = StaticModelForClassification.from_pretrained(model_name=check_model)
    hf_tokenizer = check_classifier.tokenizer
except Exception:
    hf_tokenizer = None
    print("Could not load classifier for tokenizer comparison — skip this cell.")

if hf_tokenizer is not None:
    tok_dir = EXPORT_DIR / check_model.split("/")[-1]
    if (tok_dir / "tokenizer.json").exists():
        mini_tok = MinimalTokenizer(str(tok_dir / "tokenizer.json"))
        print(f"Tokenizer type: {mini_tok.model_type}")

        sample_texts = test_data["sentence"][:20]
        mismatches = 0
        for text in sample_texts:
            hf_ids = hf_tokenizer.encode(text, add_special_tokens=False).ids
            mini_ids = mini_tok.encode(text)
            if hf_ids != mini_ids:
                mismatches += 1
                print(f"MISMATCH: '{text[:60]}…'")
                print(f"  HF:   {hf_ids[:15]}")
                print(f"  Mini: {mini_ids[:15]}")

        if mismatches == 0:
            print(f"All {len(sample_texts)} samples match!")
        else:
            print(f"\n{mismatches}/{len(sample_texts)} mismatches.")
    else:
        print(f"No tokenizer.json found at {tok_dir} — was this model exported?")

    del check_classifier
    gc.collect()

Tokenizer type: Unigram
All 20 samples match!


---
## 12. ONNX Test-Set Accuracy

Quick end-to-end check: run the full test set through the ONNX classifier and
compare to ground truth.

In [25]:
best_model = MODELS[-1]
best_onnx_dir = EXPORT_DIR / best_model.split("/")[-1]

clf_onnx = OnnxClassifier(str(best_onnx_dir))

batch_size = 256
all_preds = []
for i in range(0, len(test_data["sentence"]), batch_size):
    batch = test_data["sentence"][i : i + batch_size]
    all_preds.extend(clf_onnx.predict(batch))

correct = sum(
    1 for pred, true in zip(all_preds, test_data["label"]) if pred == true
)
total = len(test_data["label"])
print(f"ONNX accuracy ({best_model}): {correct}/{total} = {correct/total:.4f}")

ONNX accuracy (minishlab/potion-multilingual-128M): 1729/1828 = 0.9458


---
## 13. Export Metadata

In [26]:
meta = {
    "task": "intent-classification",
    "language": LANG,
    "dataset": DATASET_NAME,
    "columns": {
        "sentence": SENTENCE_COL,
        "label": LABEL_COL,
        "lang": LANG_COL,
    },
    "split_ratios": {
        "train": round(1 - TEST_RATIO - VAL_RATIO, 4),
        "validation": VAL_RATIO,
        "test": TEST_RATIO,
    },
    "split_sizes": {
        "train": len(train_data["sentence"]),
        "validation": len(val_data["sentence"]),
        "test": len(test_data["sentence"]),
    },
    "num_labels": len(set(train_data["label"])),
    "seed": SEED,
    "models": {},
    "exported": datetime.datetime.utcnow().isoformat() + "Z",
}

for model_name, result in RESULTS.items():
    parsed = result["parsed"]
    meta["models"][model_name] = {
        "accuracy": parsed.get("accuracy", 0),
        "macro_f1": parsed.get("macro avg", {}).get("f1-score", 0),
        "onnx_dir": result["onnx_dir"],
    }

meta_path = EXPORT_DIR / "metadata.json"
meta_path.write_text(json.dumps(meta, indent=2))
print(f"Metadata written to {meta_path}")
print(json.dumps(meta, indent=2))

Metadata written to exported_models_pt-PT/metadata.json
{
  "task": "intent-classification",
  "language": "pt-PT",
  "dataset": "OpenVoiceOS/MT-intents-dataset-pt-PT",
  "columns": {
    "sentence": "sentence",
    "label": "label",
    "lang": "lang"
  },
  "split_ratios": {
    "train": 0.7,
    "validation": 0.15,
    "test": 0.15
  },
  "split_sizes": {
    "train": 8488,
    "validation": 1828,
    "test": 1828
  },
  "num_labels": 157,
  "seed": 42,
  "models": {
    "Jarbas/m2v-256-bert-base-portuguese-cased": {
      "accuracy": 0.94,
      "macro_f1": 0.81,
      "onnx_dir": "exported_models_pt-PT/m2v-256-bert-base-portuguese-cased"
    },
    "Jarbas/m2v-256-bert-large-portuguese-cased": {
      "accuracy": 0.94,
      "macro_f1": 0.79,
      "onnx_dir": "exported_models_pt-PT/m2v-256-bert-large-portuguese-cased"
    },
    "Jarbas/m2v-256-albertina-100m-portuguese-ptpt-encoder": {
      "accuracy": 0.93,
      "macro_f1": 0.76,
      "onnx_dir": "exported_models_pt-PT/m2v-2